In [29]:
import pandas as pd
import numpy as np
import random
import math
from itertools import permutations

In [30]:
# COUNT Completion Time of sequence of order in flowshop schedule
def count_completion_time(processing_times):
    process_accumlutaion = np.zeros(processing_times.shape)
    accumulation_time = 0
    for i in range(len(processing_times[0])):
        accumulation_time += processing_times[0][i]
        process_accumlutaion[0][i] = accumulation_time

    for i in range(1, len(processing_times)):
        accumulation_time = process_accumlutaion[i-1][0]
        for j in range(len(processing_times[0])):
            if(j == 0):
                accumulation_time += processing_times[i][j]
            else:
                if(process_accumlutaion[i-1][j] < process_accumlutaion[i][j-1]):
                    accumulation_time += processing_times[i][j]
                else:
                    accumulation_time = process_accumlutaion[i-1][j]
                    accumulation_time += processing_times[i][j]
            process_accumlutaion[i][j] = accumulation_time
    return accumulation_time

In [31]:
def generate_random_sequence():
  """Generates a random sequence of numbers from 1 to 6."""
  sequence = random.sample(range(1, 7), 6)  # Sample 6 unique numbers from range 1 to 6
  return ''.join(map(str, sequence))  # Convert sequence to string with '-' separator


In [60]:
def sequence_to_string(sequence):
  """Converts a sequence of numbers to a string.
  Args:
      sequence: A tuple of integers representing the sequence.

  Returns:
      A string representation of the sequence.
  """
  return ''.join(map(str, sequence))

In [65]:

def generate_all_possibilites(data):
    numbers = range(1, 7)  # Create a sequence from 1 to 6

    # Use permutations to generate all possible orderings
    all_sequences = list(permutations(numbers))

    # Convert to List of String
    string_sequences = [sequence_to_string(seq) for seq in all_sequences]
    for i in range(len(string_sequences)):
        seq_list = np.zeros(data.shape)
        for j in range(len(seq)):
            seq_list[i] = data[int(seq[i])-1]
        sol.update({seq: count_completion_time(seq_list)})
    return sol



<class 'range'>


In [66]:
# Generate Random Solution of amount_of_solution in return of dictionary with data
# {"sequence" : "compleiton_time"}
# ...
def generate_random_solution(data, amount_of_solution:int):
    sol = dict()
    for x in range(amount_of_solution):
        seq = generate_random_sequence()
        seq_list = np.zeros(data.shape)
        for i in range(len(seq)):
            seq_list[i] = data[int(seq[i])-1]
        sol.update({seq: count_completion_time(seq_list)})
    return sol


In [33]:
def min_max_normalization(arr):
    min_val = np.min(arr) -1 # (np.min(arr)  * 1/100)
    max_val = np.max(arr)
    return (arr - min_val) / (max_val - min_val)

In [34]:
# Z-Score Normalization 
def zscore_normalization(arr):
    mean_val = np.mean(arr)
    std_dev = np.std(arr)
    return (arr - mean_val) / std_dev


In [35]:
def find_up_low_bound(arr):
    max_val = np.max(arr)
    min_val = np.min(arr)
    up_bound = math.ceil(max_val)
    low_bound = math.floor(min_val)
    return low_bound, up_bound

In [36]:
def sort_by_index(data):
  """Sorts a list and returns the corresponding sorted index sequence.

  Args:
      data: A list of sortable elements.

  Returns:
      A list of indices representing the sorted order of the original list.
  """
  return sorted(range(len(data)), key=lambda i: data[i])


### RUN

In [67]:
#LOAD DATA
data = pd.read_csv("data.csv")
data = data.values
data = data[:, 1:]


In [68]:
# INIT THE PARAMETERS
population_size = 10
iteration_ = 10
gamma = 0.01
alpha = 0.1
beta = 0.2
delta = 0.97

In [69]:
# Generate Random Solution based on population_size
solution_ = generate_random_solution(data=data, amount_of_solution=population_size)

IndexError: string index out of range

In [40]:
solution_

{'416532': 291.0,
 '213465': 290.0,
 '625134': 276.0,
 '136245': 286.0,
 '654213': 281.0,
 '124356': 273.0,
 '543612': 290.0,
 '634152': 286.0,
 '124536': 273.0,
 '163425': 286.0}

In [41]:
solution_keys = list(solution_.keys())
solution_list = list(solution_.values())
solution_list = np.array(solution_list)

In [42]:

solution_keys

['416532',
 '213465',
 '625134',
 '136245',
 '654213',
 '124356',
 '543612',
 '634152',
 '124536',
 '163425']

In [43]:
normalize_solution = zscore_normalization(solution_list)

In [44]:
normalize_solution

array([ 1.17376207,  1.02327975, -1.08347268,  0.42135049, -0.3310611 ,
       -1.53491963,  1.02327975,  0.42135049, -1.53491963,  0.42135049])

In [45]:
low_bound, up_bound  = find_up_low_bound(normalize_solution)

In [46]:
# CREATE OBJECT AND RUN ALGORITHM
from firefly import firefly_algo

In [47]:
fa = firefly_algo(data=normalize_solution, population=population_size, max_iter=iteration_, low_bound=low_bound, up_bound=up_bound, gamma=gamma, alpha=alpha, beta=beta, delta=delta)
fa.run()

In [48]:
fa.print_result()

1.174
1.023
-21.616
0.421
-0.331
-22.344
1.023
0.421
-20.424
0.421


In [49]:
sorted_indices = sort_by_index(normalize_solution)

In [50]:
res = dict()
for i in sorted_indices:
    res.update({solution_keys[i]: fa.data[i]})

In [51]:
res

{'124356': -22.34405568707147,
 '625134': -21.61627191589022,
 '124536': -20.42379437050973,
 '654213': -0.3310610959858636,
 '136245': 0.4213504858001939,
 '634152': 0.4213504858001939,
 '163425': 0.4213504858001939,
 '213465': 1.02327975122904,
 '543612': 1.02327975122904,
 '416532': 1.1737620675862515}